In [ ]:
from imports import *
from chargeSteps import chargeSteps
import numpy as np

In [ ]:
# SuperSim V16-00-00 + elog/2125 runs (pick one!)
##data = chargeSteps('data/EPot_highV/V16-00-00/100eh', 'HV100mm')
##data = chargeSteps('data/EPot_highV/V16-00-00/100eh-HV100mmSi', 'HV100mmSi')
##data = chargeSteps('data/EPot_highV/V16-00-00/100eh-HVeV', 'HVeV')

In [ ]:
# New TrackLimiter runs (G4CMP-358)
##data = chargeSteps('data/EPot_highV/G4CMP358', 'HV100mm', '-max*k')
data = chargeSteps('data/EPot_highV/G4CMP358', 'iZIP5', '-max*k',
                   volts=[1,4,10,25,50,75,100,200,300,400,500,600,700])

In [ ]:
data.limitVoltage(0,150)

In [ ]:
# SuperSim develop runs (noLimits)
##data = chargeSteps('data/EPot_highV/noLimits', 'HV100mm')
##data = chargeSteps('data/EPot_highV/noLimits', 'iZIP5')
##data = chargeSteps('data/EPot_highV/noLimits', 'HVeV')

In [ ]:
# New intervalley scattering code (G4CMP-404)
##data = chargeSteps('data/EPot_highV/G4CMP404','HV100mm','-newIV')

In [ ]:
# Fix to TimeStepper for stopping voltage (G4CMP-482)
##data = chargeSteps('data/EPot_highV/G4CMP482','HV100mm','-stepperFix')

# Apply voltage dependent step limit, in G4CMP-482 branch
##data = chargeSteps('data/EPot_highV/G4CMP482','HV100mm','-max*k')

In [ ]:
data.StepStack("E", log=False)

In [ ]:
data.StepStack("U", log=False)

In [ ]:
data.PathLengths("E", 10)

In [ ]:
data.PathLengths("E", data.vmax)

In [ ]:
data.PathLengths("U", data.vmax)

In [ ]:
#data.PathRatio("E", data.vmax)

In [ ]:
#data.PathRatio("U", data.vmax)

In [ ]:
#data.PathVsSteps("E", 200)

In [ ]:
#data.PathVsSteps("E", data.vmax)

In [ ]:
#data.PathVsSteps("U", data.vmax)

In [ ]:
data.StepsVsFinalRZ("E", 10)

In [ ]:
data.StepsVsFinalRZ("E", 100)

In [ ]:
#data.StepsVsFinalRZ("E", 200)

In [ ]:
#data.StepsVsFinalRZ("E", 300)

In [ ]:
#data.StepsVsFinalRZ("E", 400)

In [ ]:
data.StepsVsFinalRZ("E", data.vmax)

In [ ]:
data.StepsVsFinalRZ("U", 10)

In [ ]:
#data.StepsVsFinalRZ("U", 100)

In [ ]:
#data.StepsVsFinalRZ("U", 200)

In [ ]:
#data.StepsVsFinalRZ("U", 500)

In [ ]:
#data.StepsVsFinalRZ("U", 800)

In [ ]:
data.StepsVsFinalRZ("U", data.vmax)

In [ ]:
# For iZIP5, why is 1000V data point at same maxSteps as 900V?
maxUhit = np.array(data.StepsVsVData("U"))/1e3    # Fit in thousands
curve = np.polynomial.Polynomial((100,0,0.1,-0.001,2.5e-6))   # Initial guess
vrange = (data.vmin,data.vmax)
quad = curve.fit(data.voltage[:-1], maxUhit[:-1], 4, domain=[])
print(quad)
data.StepsVsVoltage(curve=quad*1e3,suffix="FitUniform")
maxUhit=None
curve=None
quad=None

In [ ]:
coeffK = (1200.,-60.,0.65,-1.8e-3,1.85e-6)       # Rounded, for kilosteps
fit = 1e3*np.polynomial.Polynomial(coeffK,window=vrange)
data.StepsVsVoltage(curve=fit,suffix="Fit2Uniform")

In [ ]:
# Bounding coefficient attempt for iZIP5
upperK = (1200.,0,0.6,-0.0001,1e-6)
bound = 1e3*np.polynomial.Polynomial(upperK,vrange,window=vrange)
data.StepsVsVoltage(curve=bound,suffix="Bound")

In [ ]:
data.StepsVsVoltage(stepcut=1e7,curve=bound)

In [ ]:
# Make consolidated plots overlaying three detector types

thedir = 'data/EPot_highV/noLimits'
snolab = chargeSteps(thedir, 'HV100mm', volts=(1,4,10,25,50,75,100,200))
soudan = chargeSteps(thedir, 'iZIP5', volts=(1,4,10,25,50,75,100,200))
hvev   = chargeSteps(thedir, 'HVeV')

In [ ]:
snolab.PlotStepsVsVoltage("U")
soudan.PlotStepsVsVoltage("U")
hvev.PlotStepsVsVoltage("U")
plt.title("Charged Tracks, Uniform Field Model")
plt.legend()
plt.savefig("MaxSteps-Voltage_allDet.png")

In [ ]:
snolab.PlotStepsVsField("U")
soudan.PlotStepsVsField("U")
hvev.PlotStepsVsField("U")
plt.xlim(0,20000)
plt.title("Charged Tracks, Uniform Field Model")
plt.legend()
plt.savefig("MaxSteps-EField_allDet.png")